In [ ]:
!pwd

import sys
import os

# Detect if we are in Colab or a generic cloud environment (like Vast.ai)
is_colab = 'google.colab' in sys.modules 
is_vast = os.path.exists('/root/.vast_container') # Common Vast.ai indicator
is_cloud = is_colab or is_vast

if is_cloud:
  py_url = "https://raw.githubusercontent.com/fanaustinca/so101ai/refs/heads/main/lerobot_util.py"

  !curl -O {py_url}
  if os.path.exists("lerobot_setup.py"):
    print("\nSuccessfully downloaded lerobot_setup.py")
  else:
    print("\nDownload failed.")
else:
  print("Running on local machine. Skipping cloud setup.")

In [ ]:
%conda install python-dotenv -y

# Setting lerobot env
from lerobot_util import setup_lerobot_env

setup_lerobot_env()

In [ ]:
# Run the following command to identify USB port for arms
from lerobot_util import print_shell_md
import sys

print_shell_md(
  "Run following command to identify USB port for arms.", 
  f"{sys.executable}",
  "src/lerobot/scripts/lerobot_find_port.py"
)

print_shell_md(
  "Run following command to identify camera index.", 
  f"{sys.executable}",
  "lerobot/src/lerobot/scripts/lerobot_find_cameras.py"
)


In [ ]:
import sys
import json

from lerobot_util import print_shell_md

leader_port = '/dev/ttyACM0'
follower_port = '/dev/ttyACM1'
wrist_camera_index = 2
top_camera_index = 4
hf_namespace = 'fanaustinca'
data_repo = f"{hf_namespace}/fll_training_data"

camera_config = {
  "wrist": {
    "type": "opencv", 
    "index_or_path": 2, 
    "width": 640, 
    "height": 480, 
    "fps": 30
  },
  "top": {
    "type": "opencv", 
    "index_or_path": 4, 
    "width": 640, 
    "height": 480, 
    "fps": 30
  }
}

camera_config_str = json.dumps(camera_config, separators=(',', ':'))

arguments = [
    "src/lerobot/scripts/lerobot_teleoperate.py",
    "--robot.type=so101_follower",
    f"--robot.port={follower_port}",
    "--teleop.type=so101_leader",
    f"--teleop.port={leader_port}",
    f"--robot.cameras=\'{camera_config_str}\'",
    "--display_data=true"
]

print_shell_md(
  "Run following command to operate leader arm to control follower arm.", 
  f"{sys.executable}",
  *arguments
)

arguments = [
  "src/lerobot/scripts/lerobot_record.py",
  "--robot.type=so101_follower",
  f"--robot.port={follower_port}"
  f"--robot.cameras=\'{camera_config_str}\'",
  "--teleop.type=so101_leader",
  f"--teleop.port={leader_port}"
  f"--dataset.repo_id={data_repo}",
  "--dataset.num_episodes=50",
  "--dataset.single_task=fll_collection",
  "--dataset.push_to_hub=true",
  "--dataset.private=false"
]

print_shell_md(
  "Run following command to collect training data.", 
  f"{sys.executable}",
  *arguments
)


In [ ]:
from lerobot_util import cd_lerobot
import json
import sys
from IPython.display import display, Markdown

cd_lerobot()

# Training ACT model.
# Adjust the following variables before starting training
tag = '400episodes'
policy_type = 'diffusion'
n_action_steps = 10 if policy_type == 'diffusion' else 100
hf_namespace = 'fanaustinca'
training_repo = f"{hf_namespace}/fll_training_combined"
model_repo = f"{hf_namespace}/fll_diffusion_model_400episodes_v2"
# Use 'local_output' consistently
local_output = f"outputs/model/{model_repo}"
steps = 150000
batch_size = 64
num_workers = 16
enable_image_transforms = True

if policy_type not in ['act', 'diffusion']:
    raise ValueError(f"policy_type can only be 'act' or 'diffusion', got: {policy_type}")


# Clean up output from last run
!rm -rf {local_output}

tfs_config = {
  "to_tensor": {"type": "ToImage"},
  "affine": {
    "type": "RandomAffine",
    "kwargs": {"degrees": 0, "translate": [0.05, 0.05]}
  }
}

"""
    --dataset.image_transforms.enable=true \
    --dataset.image_transforms.random_order=false \
    --dataset.image_transforms.tfs='{tfs_string}' \
"""

# Convert to a compact JSON string for the CLI
tfs_string = json.dumps(tfs_config, separators=(',', ':'))

arguments = f"""\
    --policy.type={policy_type} \
    --dataset.repo_id={training_repo} \
    --job_name=fll_training_{policy_type} \
    --output_dir={local_output} \
    --policy.n_action_steps={n_action_steps} \
    --policy.device=cuda \
    --steps={steps} \
    --save_freq=5000 \
    --batch_size={batch_size} \
    --log_freq=100 \
    --num_workers={num_workers} \
    --policy.use_amp=true \
    --policy.push_to_hub=true \
    --policy.repo_id={model_repo} \
    --dataset.image_transforms.enable=true \
    --dataset.image_transforms.random_order=true \
    --wandb.enable=true"""

if enable_image_transforms:
    arguments += " --dataset.image_transforms.enable=true \
      --dataset.image_transforms.random_order=true"

if policy_type == 'diffusion':
    arguments += " --policy.num_inference_steps=30"

# Your variables
command_str = f"{sys.executable} src/lerobot/scripts/lerobot_train.py {arguments}"

# Wrap it in markdown code blocks
display(Markdown(f"```bash\n{command_str}\n```"))

command_str = f"hf repo tag create {model_repo} {tag} -m \"Diffusion model trained on {training_repo}, tag {tag}\""
display(Markdown(f"```bash\n{command_str}\n```"))
